In [1]:
import re
import tweepy
import pandas as pd
from tweepy import OAuthHandler
import yaml
# from http.client import IncompleteRead
# import threading 
# import datetime 
    
    
with open('credential1.yaml') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    
consumer_key = config['consumer_key']
consumer_secret = config['consumer_secret']
access_token = config['access_token']
access_secret = config['access_secret']
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#### Collect tweets with hashtag `#30mapchallenges`

In [4]:
def collect_tweets(search_words, since):
    #tweets
    tweet_id = []
    tweet_text = []
    language = []
    tweet_created_at = []
    retweet_count = []
    favorite_count= []
    hashtags= []
    #users
    user_id = []
    user_name = []
    user_screen_name = []
    user_loc = []
    user_follower = []
    friends_count = []
    #media
    media = []
    for tweet in tweepy.Cursor(api.search,
                               q=search_words,
                               since= since, 
                               include_entities=True, 
                               tweet_mode="extended").items():
        #tweets
        tweet_id.append(tweet.id)
        tweet_text.append(tweet.full_text)
        language.append(tweet.lang)
        tweet_created_at.append(tweet.created_at)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
        try:
            hashtags.append(tweet.entities.get('hashtags', []))
        except KeyError:
            pass
#         hashtags.append(tweet.entities.hashtags)
        #users
        user_id.append(tweet.user.id)
        user_name.append(tweet.user.name)
        user_screen_name.append(tweet.user.screen_name)
        user_loc.append(tweet.user.location)
        user_follower.append(tweet.user.followers_count)
        friends_count.append(tweet.user.friends_count)
        #media
        try:
            media.append(tweet.entities.get('media')[0].get('media_url'))
#             print(tweet.entities.get('media'))
        except Exception as e:
            media.append(None)
        #print([tweet.user.name,tweet.created_at,tweet.text])
    df = pd.DataFrame({'tweet_id':tweet_id,
                        'tweet_text':tweet_text,
                        'language':language,
                        'tweet_created_at':tweet_created_at,
                        'retweet_count':retweet_count,
                        'favorite_count':favorite_count,
                        'hashtags':hashtags,
                        'user_id':user_id,
                        'user_name':user_name,
                        'user_screen_name':user_screen_name,
                        'user_loc':user_loc,
                        'user_follower':user_follower,
                        'friends_count':friends_count,
                        'media':media})
            
    return df

In [50]:
search_words = "#30DayMapChallenge -filter:retweets"
since = "2020-10-30"

df = collect_tweets(search_words, since)

df = df[(df.tweet_created_at >= '2020-11-01 00:00:00') & 
   (df.tweet_created_at <= '2020-11-02 00:00:00')]

# df1 = df[df['tweet_text'].str.contains("points") | df['tweet_text'].str.contains("day 1")]

df = df[df['media'].notnull()]
df['tweet_url'] = df['tweet_text'].str.extract('(?P<url>https?://[^\s]+)', expand=False).str.strip()


# df.to_csv(f"./data/day1.csv")

In [51]:
df

,tweet_id,tweet_text,language,tweet_created_at,retweet_count,favorite_count,hashtags,user_id,user_name,user_screen_name,user_loc,user_follower,friends_count,media,tweet_url
1148,1323051948485070853,Este mes estaré participando en el #30DayMapCh...,es,2020-11-01 23:59:00,1,10,"[{'text': '30DayMapChallenge', 'indices': [35,...",1002117891133755392,José Molano-Pulido,JoseFMolano,,261,437,http://pbs.twimg.com/media/ElxsT-SWkAAhiSj.png,https://t.co/Qw7z5xS2Qh
1151,1323046291367100416,Antes de que acabe el día. Uno de mis primero...,es,2020-11-01 23:36:31,16,72,"[{'text': '30DayMapChallenge', 'indices': [83,...",384907052,Leticia Luna,Letyludigital,México,1121,2447,http://pbs.twimg.com/media/ElxnlwaVcAEq9yb.jpg,https://t.co/KXhtzKjh2e
1152,1323043516629614595,We'll see if I stick with this #30DayMapChalle...,en,2020-11-01 23:25:29,2,15,"[{'text': '30DayMapChallenge', 'indices': [31,...",704329847896678400,Ross Thorn,RealRossThorn,Minnesota,323,482,http://pbs.twimg.com/media/Elxk6HSXEBg-O8d.jpg,https://t.co/QGuHH8qogZ
1153,1323042272146034689,Day 1 of #30DayMapChallenge - Points\n\nQuickl...,en,2020-11-01 23:20:33,3,11,"[{'text': '30DayMapChallenge', 'indices': [9, ...",863722747,Tim Rodaway,TimRodaway,"Newcastle Upon Tyne, England",94,283,http://pbs.twimg.com/media/ElxjSlZW0AIxSnq.jpg,https://t.co/yPIDpPpSIJ
1154,1323042145255698449,#30DayMapChallenge day 1 Points\nFirst attempt...,en,2020-11-01 23:20:03,1,2,"[{'text': '30DayMapChallenge', 'indices': [0, ...",3641471067,Bert H,BertrandHoud,"Auvergne, France",64,259,http://pbs.twimg.com/media/ElxjxjNXUAAVWUC.jpg,https://t.co/0hCmCUWb5a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519,1322756313948938243,Day 1 of #30DayMapChallenge - Points.\n\nI vis...,en,2020-11-01 04:24:15,3,13,"[{'text': '30DayMapChallenge', 'indices': [9, ...",47001569,ぞあ.,k_zoar,ジオイドと地表の狭間,827,750,http://pbs.twimg.com/media/Eltfw1rU4AAOAdi.jpg,https://t.co/5aG8IneLpH
1520,1322748863854206977,Day 1 of #30DayMapChallenge. Points.\n\nA love...,en,2020-11-01 03:54:39,1,19,"[{'text': '30DayMapChallenge', 'indices': [9, ...",29162437,Kerrin Sime,kjsime,"NZ, most of the time.",38,118,http://pbs.twimg.com/media/EltZFJcVkAERA-J.jpg,https://t.co/NGaOJr4CLF
1522,1322710477462360064,Day 1 Points\nEarthquakes from 1959 to 2020\nD...,en,2020-11-01 01:22:07,3,36,"[{'text': '30DayMapChallenge', 'indices': [89,...",1056888822942711808,mits miyake,mmits003,Sapporo,171,361,http://pbs.twimg.com/tweet_video_thumb/ElsykJK...,https://t.co/I9Bev7Oo5y
1523,1322704397013377025,Day 1 of #30DayMapChallenge - Points\n\nHigh P...,en,2020-11-01 00:57:57,12,153,"[{'text': '30DayMapChallenge', 'indices': [9, ...",93839237,Andrew DC,andrewdc_nz,"Christchurch City, New Zealand",404,165,http://pbs.twimg.com/media/ElswRHqU8AEPkqJ.jpg,https://t.co/fcXAJxTLKo


## Overview

In [188]:
# df = pd.read_csv("./data/day1.csv")

In [122]:
df.user_loc

163                     
164    Neo SCL / Neo BCN
165                 Wien
166     Montréal, Québec
169                     
             ...        
704                     
705                  y,x
706        Leicester, UK
707                     
708                 Iraq
Name: user_loc, Length: 456, dtype: object

In [121]:
df[['retweet_count','favorite_count', 'user_follower', 'friends_count']].describe()

,retweet_count,favorite_count,user_follower,friends_count
count,456.000000,456.000000,456.000000,456.000000
mean,2.923246,14.309211,1602.668860,668.684211
std,5.005121,30.491772,6699.908748,852.130466
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,3.000000,69.000000,154.750000
50%,1.000000,7.000000,269.000000,361.000000
75%,3.000000,16.000000,844.000000,831.500000
max,82.000000,546.000000,80339.000000,5446.000000


## Top favorited `#30mapchallenges` post

In [61]:
df_fav15 = df.sort_values(by='favorite_count', ascending=False).head(15)
df_fav15 = df_fav15[['user_screen_name', 'retweet_count', 'favorite_count', 'media']]
df_fav15['user_screen_name'] = '**[@' + df['user_screen_name'] +']('+ df['tweet_url']+')**'
df_fav15['media'] = '![](' + df['media'] +')'

In [57]:
from tabulate import tabulate

print(tabulate(df_fav15, headers='keys', tablefmt='github'))

|      | user_screen_name                               |   retweet_count |   favorite_count | media                                                                                        |
|------|------------------------------------------------|-----------------|------------------|----------------------------------------------------------------------------------------------|
| 1355 | **[@Juanizio_C](https://t.co/7NtevAqkNd)**     |             274 |              497 | ![](http://pbs.twimg.com/media/Elv5tUaW0AA4CT-.jpg)                                          |
| 1379 | **[@rajbhagatt](https://t.co/0tQIrXrU0G)**     |              90 |              390 | ![](http://pbs.twimg.com/media/ElvsxU5VgAANXV5.jpg)                                          |
| 1511 | **[@lacxrx](https://t.co/0WeTc3o9Bw)**         |              54 |              340 | ![](http://pbs.twimg.com/media/EluFSpkWkAES1Tc.jpg)                                          |
| 1348 | **[@allyoucanmap](https://t.co/M4gV6u1Y6W

## Top retweeted `#30mapchallenges` post

In [62]:
df_rt15 = df.sort_values(by='retweet_count', ascending=False).head(15)
df_rt15 = df_rt15[['user_screen_name', 'retweet_count', 'favorite_count', 'media']]
df_rt15['user_screen_name'] = '**[@' + df['user_screen_name'] +']('+ df['tweet_url']+')**'
df_rt15['media'] = '![](' + df['media'] +')'


In [59]:
print(tabulate(df_rt15, headers='keys', tablefmt='github'))

|      | user_screen_name                               |   retweet_count |   favorite_count | media                                                                                        |
|------|------------------------------------------------|-----------------|------------------|----------------------------------------------------------------------------------------------|
| 1355 | **[@Juanizio_C](https://t.co/7NtevAqkNd)**     |             274 |              497 | ![](http://pbs.twimg.com/media/Elv5tUaW0AA4CT-.jpg)                                          |
| 1379 | **[@rajbhagatt](https://t.co/0tQIrXrU0G)**     |              90 |              390 | ![](http://pbs.twimg.com/media/ElvsxU5VgAANXV5.jpg)                                          |
| 1511 | **[@lacxrx](https://t.co/0WeTc3o9Bw)**         |              54 |              340 | ![](http://pbs.twimg.com/media/EluFSpkWkAES1Tc.jpg)                                          |
| 1503 | **[@OrdnanceSurvey](https://t.co/oPBbiF3x

### Late entry

In [60]:
df_late = pd.read_csv("./data/day1_late.csv")

In [68]:
df_late_fav = df_late.sort_values(by='favorite_count', ascending=False).head(5)
df_late_fav = df_late_fav[['user_screen_name', 'retweet_count', 'favorite_count', 'media']]
df_late_fav['user_screen_name'] = '**[@' + df_late['user_screen_name'] +']('+ df_late['tweet_url']+')**'
df_late_fav['media'] = '![](' + df_late['media'] +')'
df_late_fav

,user_screen_name,retweet_count,favorite_count,media
42,**[@darenw](https://t.co/rZ2gtbJmI4)**,4,95,![](http://pbs.twimg.com/media/Elyc4d4XUAEs09h...
50,**[@gonzalotalavera](https://t.co/I6UNoGYVOx)**,22,60,![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2...
53,**[@PratapVardhan](https://t.co/WTT8WM17qy)**,4,43,![](http://pbs.twimg.com/media/Elx5722WkAEOOMf...
2,**[@tomasral](https://t.co/khysneCwjM))**,5,35,![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf...
54,**[@BZgeo](https://t.co/LavH2067O2)**,3,26,![](http://pbs.twimg.com/media/Elx0-fPXIAc9JGJ...


In [69]:
print(tabulate(df_late_fav, headers='keys', tablefmt='github'))

|    | user_screen_name                                |   retweet_count |   favorite_count | media                                               |
|----|-------------------------------------------------|-----------------|------------------|-----------------------------------------------------|
| 42 | **[@darenw](https://t.co/rZ2gtbJmI4)**          |               4 |               95 | ![](http://pbs.twimg.com/media/Elyc4d4XUAEs09h.jpg) |
| 50 | **[@gonzalotalavera](https://t.co/I6UNoGYVOx)** |              22 |               60 | ![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg) |
| 53 | **[@PratapVardhan](https://t.co/WTT8WM17qy)**   |               4 |               43 | ![](http://pbs.twimg.com/media/Elx5722WkAEOOMf.jpg) |
|  2 | **[@tomasral](https://t.co/khysneCwjM))**       |               5 |               35 | ![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf.jpg) |
| 54 | **[@BZgeo](https://t.co/LavH2067O2)**           |               3 |               26 | ![](http://pbs.twi

In [70]:
df_late_rt = df_late.sort_values(by='retweet_count', ascending=False).head(5)
df_late_rt = df_late_rt[['user_screen_name', 'retweet_count', 'favorite_count', 'media']]
df_late_rt['user_screen_name'] = '**[@' + df_late['user_screen_name'] +']('+ df_late['tweet_url']+')**'
df_late_rt['media'] = '![](' + df_late['media'] +')'
df_late_rt

,user_screen_name,retweet_count,favorite_count,media
50,**[@gonzalotalavera](https://t.co/I6UNoGYVOx)**,22,60,![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2...
24,**[@neocartocnrs](https://t.co/1GcTGraT9T)**,8,14,![](http://pbs.twimg.com/media/Elz2OvNXEAAItfe...
2,**[@tomasral](https://t.co/khysneCwjM))**,5,35,![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf...
57,**[@nurussadad](https://t.co/z4HaIfQMBi)**,4,7,![](http://pbs.twimg.com/media/ElxuzgpU0AEIbdT...
7,**[@geologyjesse](https://t.co/dR68mE4zhD)**,4,14,![](http://pbs.twimg.com/media/El1vroKWoAEozEk...


In [71]:
print(tabulate(df_late_rt, headers='keys', tablefmt='github'))

|    | user_screen_name                                |   retweet_count |   favorite_count | media                                               |
|----|-------------------------------------------------|-----------------|------------------|-----------------------------------------------------|
| 50 | **[@gonzalotalavera](https://t.co/I6UNoGYVOx)** |              22 |               60 | ![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg) |
| 24 | **[@neocartocnrs](https://t.co/1GcTGraT9T)**    |               8 |               14 | ![](http://pbs.twimg.com/media/Elz2OvNXEAAItfe.jpg) |
|  2 | **[@tomasral](https://t.co/khysneCwjM))**       |               5 |               35 | ![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf.jpg) |
| 57 | **[@nurussadad](https://t.co/z4HaIfQMBi)**      |               4 |                7 | ![](http://pbs.twimg.com/media/ElxuzgpU0AEIbdT.png) |
|  7 | **[@geologyjesse](https://t.co/dR68mE4zhD)**    |               4 |               14 | ![](http://pbs.twi

### Send tweet via API

In [3]:
api.update_status("Look, I'm tweeting from #Python")


Status(_api=<tweepy.api.API object at 0x0000011E60202A48>, _json={'created_at': 'Mon Nov 02 22:53:36 +0000 2020', 'id': 1323397878983856130, 'id_str': '1323397878983856130', 'text': "Look, I'm tweeting from #Python", 'truncated': False, 'entities': {'hashtags': [{'text': 'Python', 'indices': [24, 31]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://hn303.github.io/CamLandEc-RM03/" rel="nofollow">Lonon_REST</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 528087151, 'id_str': '528087151', 'name': 'Haifeng Niu', 'screen_name': 'niu_haifeng', 'location': 'Cambridge, England', 'description': 'PhD candidate in Lab of Interdisciplinary Spatial Analysis, Department of Land Economy, University of Cambridge @CambridgeLandEc', 'url': 'https://t.co/nuaPhizg9Y', 'entities': {'url': {'urls': [{'url': 'https://t.co/nuaPhizg9Y', 'expanded_